Import packages

In [9]:
import os
import pickle
import sys
import tarfile

from PIL import Image
from six.moves.urllib.request import urlretrieve

import argparse
import math
import time

%matplotlib inline
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import numpy.random as npr
import scipy.misc
import torch
from torchvision.transforms.functional import InterpolationMode
from torchvision.transforms import Compose, Resize, ToTensor
import torchvision.transforms as T
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable

ModuleNotFoundError: No module named 'torch'

Utility functions

In [8]:
# Download dataset
origin = "https://www.eecs.berkeley.edu/Research/Projects/CS/vision/grouping/BSR/BSR_bsds500.tgz"
f = "BSR_bsds500.tar.gz"

try:
    try:
        urlretrieve(origin, f)
    except URLError as e:
        raise Exception(error_msg.format(origin, e.errno, e.reason))
    except HTTPError as e:
        raise Exception(error_msg.format(origin, e.code, e.msg))
except (Exception, KeyboardInterrupt) as e:
    if os.path.exists(f):
        os.remove(f)
    raise

with tarfile.open(f) as archive:
    archive.extractall(".")

os.remove(f)

path = "BSR/BSDS500/data/images"

In [23]:
class BSDS500Dataset(torch.utils.data.Dataset):
    def __init__(self, path: str, mode: str, scale: int):
        self.path = path
        self.mode = mode # train / test / val
        self.scale = scale
        self.file_names = [os.path.join(self.path, self.mode, file_name) for file_name in os.listdir(os.path.join(self.path, self.mode)) if file_name.endswith(".jpg")]

    def __len__(self):
        return len(self.file_names)

    def __getitem__(self, index: int):
        file_name = self.file_names[index]
        image = Image.open(file_name)

        # Convert yo YUV and split the channels
        image.convert("YCbCr")
        y, u, v = image.split()

        low_res_size = [image.size[1] // self.scale, image.size[0] // self.scale]
        high_res_size = [low_res_size[0] * self.scale, low_res_size[1] * self.scale]

        low_composed = Compose([
            Resize(low_res_size, interpolation=InterpolationMode.BICUBIC),
            ToTensor()
        ])

        high_composed = Compose([
            Resize(high_res_size, interpolation=InterpolationMode.BICUBIC),
            ToTensor()
        ])

        sample = { "y_low": low_composed(y).cuda(), "u_low": low_composed(u).cuda(), "v_low": low_composed(v).cuda(),
               "y_high": high_composed(y).cuda(), "u_high": high_composed(u).cuda(), "v_high": high_composed(v).cuda(),
               "high": high_composed(image).cuda() }
        return sample

trainset = BSDS500Dataset(path, "train", 3)
testset = BSDS500Dataset(path, "test", 3)



for i in range(len(trainset)):
    sample = trainset[i]
    print(i, sample["y_low"].size(), sample["high"].size())
    if i == 1:
        break

0 torch.Size([3, 160, 107]) torch.Size([3, 321, 481])
1 torch.Size([3, 107, 160]) torch.Size([3, 481, 321])
2 torch.Size([3, 160, 107]) torch.Size([3, 321, 481])


The actual ESPCN model

In [ ]:
class ESPCN_model(nn.Module):
    def __init__(self, layer1_channel, layer2_channel, upscale_factor):
        super().__init__()

        self.layer1 = nn.Sequential(
            nn.Conv2d(1, layer1_channel, kernel_size=5, stride=1, padding=2),
            nn.Tanh()
        )

        self.layer2 = nn.Sequential(
            nn.Conv2d(layer1_channel, layer2_channel, kernel_size=3, stride=1, padding=1),
            nn.Tanh()
        )

        self.layer3 = nn.Sequential(
            nn.ConvTranspose2d(layer2_channel, upscale_factor ** 2, kernel_size=3, stride=1, padding=1),
            nn.PixelShuffle(upscale_factor) # This function is literally build for ESPCN
        )

    def forward(self, x):
        out = self.layer1(x)
        out = self.layer2(out)
        out = self.layer3(out)
        return out

Code for training (adapted from PA2)
The training part works, at least no error, I'll check the output later

Currently I use SGD (batch size 1) just for testing

In [ ]:
from matplotlib import transforms
from torch._C import wait
class AttrDict(dict):
    def __init__(self, *args, **kwargs):
        super(AttrDict, self).__init__(*args, **kwargs)
        self.__dict__ = self

def train(args, cnn=None):
    # Set the maximum number of threads to prevent crash in Teaching Labs
    # torch.set_num_threads(5)

    # Numpy random seed
    npr.seed(args.seed)

    # Save directory
    save_dir = "outputs/" + args.experiment_name

    # INPUT CHANNEL
    num_in_channels = 1

    # LOAD THE MODEL
    if cnn is None:
        cnn = ESPCN_model(64, 32, 3)

    # LOSS FUNCTION
    criterion = nn.MSELoss()
    optimizer = torch.optim.SGD(cnn.parameters(), lr=1e-2, momentum=0.9, weight_decay=1e-4, nesterov=False)

    # DATA
    print("Loading data...")
    # (x_train, y_train), (x_test, y_test) = load_cifar10()

    # y.save("temp.jpg", "JPEG")

    # print("Transforming data...")
    # train_rgb, train_grey = process(x_train, y_train, downsize_input=args.downsize_input)
    # train_rgb_cat = get_rgb_cat(train_rgb, colours)
    # test_rgb, test_grey = process(x_test, y_test, downsize_input=args.downsize_input)
    # test_rgb_cat = get_rgb_cat(test_rgb, colours)

    # Create the outputs folder if not created already
    if not os.path.exists(save_dir):
        os.makedirs(save_dir)

    print("Beginning training ...")
    if args.gpu:
        cnn.cuda()
    start = time.time()

    train_losses = []
    valid_losses = []
    valid_accs = []
    for epoch in range(args.epochs):
        # Train the Model
        cnn.train()  # Change model to 'train' mode
        losses = []


        for i in range(len(trainset)):
          sample = trainset[i]
          input = sample["y_low"].unsqueeze(0)

          optimizer.zero_grad()

          output = cnn(input)
          # print(input.size())
          # print(output.size())

          loss = criterion(output, sample["y_high"].unsqueeze(0))
          loss.backward()
          optimizer.step()
          losses.append(loss.data.item())

        # plot training images
        # if args.plot:
        #     _, predicted = torch.max(outputs.data, 1, keepdim=True)
        #     plot(
        #         xs,
        #         ys,
        #         predicted.cpu().numpy(),
        #         colours,
        #         save_dir + "/train_%d.png" % epoch,
        #         args.visualize,
        #         args.downsize_input,
        #     )

        # plot training images
        avg_loss = np.mean(losses)
        train_losses.append(avg_loss)
        time_elapsed = time.time() - start
        print(
            "Epoch [%d/%d], Loss: %.4f, Time (s): %d"
            % (epoch + 1, args.epochs, avg_loss, time_elapsed)
        )

        # Evaluate the model
        # cnn.eval()  # Change model to 'eval' mode (BN uses moving mean/var).
        # val_loss, val_acc = run_validation_step(
        #     cnn,
        #     criterion,
        #     test_grey,
        #     test_rgb_cat,
        #     args.batch_size,
        #     colours,
        #     save_dir + "/test_%d.png" % epoch,
        #     args.visualize,
        #     args.downsize_input,
        # )

        # time_elapsed = time.time() - start
        # valid_losses.append(val_loss)
        # valid_accs.append(val_acc)
        # print(
        #     "Epoch [%d/%d], Val Loss: %.4f, Val Acc: %.1f%%, Time(s): %.2f"
        #     % (epoch + 1, args.epochs, val_loss, val_acc, time_elapsed)
        # )

    # Plot training curve
    plt.figure()
    plt.plot(train_losses, "ro-", label="Train")
    plt.plot(valid_losses, "go-", label="Validation")
    plt.legend()
    plt.title("Loss")
    plt.xlabel("Epochs")
    plt.savefig(save_dir + "/training_curve.png")

    if args.checkpoint:
        print("Saving model...")
        torch.save(cnn.state_dict(), args.checkpoint)

    return cnn

Train the model

In [ ]:
args = AttrDict()
args_dict = {
    "gpu": True,
    "valid": False,
    "checkpoint": "",
    "colours": "./data/colours/colour_kmeans24_cat7.npy",
    "model": "UNet",
    "kernel": 3,
    "num_filters": 32,
    'learn_rate':0.001,
    "batch_size": 100,
    "epochs": 10,
    "seed": 42,
    "plot": True,
    "experiment_name": "colourization_cnn",
    "visualize": False,
    "downsize_input": False,
}
args.update(args_dict)
cnn = train(args)
